# Sequence_end_Representitive_value_Trend

Sequence의 마지막날 등락 트랜드를 대표값으로 레이블링

In [2]:
import pandas as pd
import os
from shutil import copyfile, move
from pathlib import Path

In [ ]:
def removeOutput(finput):
    if(Path(finput)).is_file():
        os.remove(finput)

In [3]:
def seqEnd_trend(fname, seq_len):
    print("Creating label . . .")
    print("type : sequence_end")
    
    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)
    df.reset_index(inplace=True)
    
    outputname = "{}_label_{}.txt".format(fname[0:-4], seq_len) # ticker 명과 sequence 길이를 파일이름에 포함
    removeOutput(outputname)
        
    df['Date'] = df['Date'].map(mdates.date2num) # Y-M-D 포멧에서 num 포멧으로 변경
    
    for i in range(0, len(df)-int(seq_len)):
        tmp_df = df.iloc[i:i + int(seq_len)]  # seq_len 만큼 데이터프레임 슬라이싱
        starting = 0
        endvalue = 0
        label = ""
        starting = tmp_df["Open"].iloc[-2] # seq 마지막날의 전날 종가
        endvalue = tmp_df["Close"].iloc[-1] # seq 마지막날 종가
        tmp_rtn = endvalue / starting - 1 
          
        if tmp_rtn > 0:
            label = 1
        else:
            label = 0
        
        with open(outputname, 'a') as the_file:
            the_file.write("{}--{},{}".format(outputname, i, label))
            the_file.write("\n")
        
    print("Create label finished.")
